# **Análisis exploratorio de calidad datos meteorológicos (AEMET)**

# Índice del Análisis Exploratorio de Calidad de Datos (Landing → Trusted)

## A. Datos generales
- A.1 Número de registros totales en el dataset
- A.2 Check de duplicados (`fecha` + `indicativo`)
- A.3 Número de registros por estación (cobertura)
  - Clasificación de estaciones por cobertura:
    - Alta (>90% de días con registros)
    - Media (80–90%)
    - Baja (<80%)
  - Evaluación de si el rango alto (90–100%) es aceptable
- A.4 Evaluación de estaciones de cobertura media y baja:
  - Distribución espacial (ubicación en el territorio)
  - Distribución temporal (¿faltantes dispersos o concentrados en periodos específicos?)
- A.5 Número de registros por estación y por año (con foco en estaciones de cobertura alta)

## B. Tipos de datos y formatos
- B.1 Valores numéricos: transformación de coma decimal a punto decimal (`float`)
- B.2 Fechas: unificación en formato `YYYY-MM-DD` y validación de fechas reales y plausibles
  - Sin fechas futuras fuera del rango esperado
- B.3 Horas: unificación a formato `HH:MM`
- B.4 Normalización de texto: provincias y nombres de estaciones (mayúsculas/minúsculas, tildes)
- B.5 Validación de unidades de medida (temperatura en ºC, precipitación en mm, etc.)

## C. Completitud
- C.1 Definición de campos obligatorios (`NOT NULL`)
- C.2 Política de tratamiento de nulos (imputación vs descarte)
- C.3 Análisis de missingness: detección de concentraciones de valores nulos en periodos concretos (ej. todas las estaciones sin `tmax` en un mes → posible fallo de ingestión)
- C.4 Cobertura temporal: validación del rango de fechas y continuidad de series por estación
- C.5 Cobertura espacial: validación del inventario de estaciones (unicidad de `indicativo`, coherencia de coordenadas)

## D. Reglas de rango y consistencia
- D.1 Coherencia entre variables relacionadas:
  - `tmin ≤ tmed ≤ tmax`
  - `hrmin ≤ hrmedia ≤ hrmax`
  - `presmin ≤ presmax`
- D.2 Valores dentro de rangos físicos plausibles en España:
  - Temperatura: [–40, 50] ºC
  - Precipitación: ≥ 0
  - Humedad relativa: [0, 100]
  - Altitud: [–100, 4000]
  - Dirección de viento: [0, 360] o códigos especiales (88, 99)
- D.3 Validación del resto de campos según su rango esperado

## E. Detección de outliers
- E.1 Identificación de valores atípicos mediante técnicas estadísticas (z-score, IQR)
- E.2 Marcado de outliers (no eliminación en esta fase)
- E.3 Notas: los outliers se analizarán en la fase de modelado para determinar si corresponden a fenómenos meteorológicos extremos reales

## F. Homogeneización y enriquecimiento
- F.1 Inclusión de un campo `fuente_datos` para trazabilidad del origen
- F.2 Generación de un flag `dato_imputado` para distinguir valores originales de estimados

## Uso de Apache Spark para el análisis exploratorio

El análisis exploratorio de los datos meteorológicos requiere procesar un volumen elevado de información: **2.848.554 registros** correspondientes a observaciones diarias de múltiples estaciones a lo largo de un periodo de casi nueve años.
Ante este escenario, resulta fundamental seleccionar una tecnología que garantice eficiencia, escalabilidad y capacidad de integración con el resto de la arquitectura del *Data Lake*.

Se ha optado por emplear **Apache Spark** en lugar de alternativas como **DuckDB**.

## Configuración inicial:
 1. Creación de sesión Spark
2. Obtención del dataset

In [1]:
# Crear builder
from pyspark.sql import SparkSession

DELTA_VERSION = "3.2.0"  # compatible con Spark 3.5.x (Scala 2.12)

spark = (
    SparkSession.builder
    .appName("EDA_Calidad_Landing")
    # Extensiones Delta
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    # 👉 Añadir los JAR de Delta desde Maven Central
    .config("spark.jars.packages", f"io.delta:delta-spark_2.12:{DELTA_VERSION}")
    .config("spark.sql.shuffle.partitions", "200")
    .getOrCreate()
)

print("Spark:", spark.version)
print("Extensiones:", spark.conf.get("spark.sql.extensions"))
print("Packages:", spark.sparkContext.getConf().get("spark.jars.packages"))

25/08/21 19:07:01 WARN Utils: Your hostname, MacBook-Pro-de-Ines.local resolves to a loopback address: 127.0.0.1; using 172.20.10.3 instead (on interface en0)
25/08/21 19:07:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/inessabate/.ivy2/cache
The jars for the packages stored in: /Users/inessabate/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-371fe86e-627c-402b-9081-f47436a9fdd1;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/inessabate/PycharmProjects/climascan-general/.venv311/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 85ms :: artifacts dl 2ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-371fe86e-627c-402b-9081-f47436a9fdd1
	confs: [default]
	0 artifacts copied, 3 already retrieved (0kB/2ms)
25/08/21 19:07:01 

Spark: 3.5.1
Extensiones: io.delta.sql.DeltaSparkSessionExtension
Packages: io.delta:delta-spark_2.12:3.2.0


In [3]:
# Obtección del dataset
from pathlib import Path

# Root del proyecto: subir dos niveles desde notebooks/01_data_analytics/
PROJECT_ROOT = Path().resolve().parents[1]
LANDING_PATH = PROJECT_ROOT / "data" / "landing" / "aemet_deltalake"

try:
    df = spark.read.format("delta").load(str(LANDING_PATH))   # si hay _delta_log
except Exception:
    df = spark.read.parquet(str(LANDING_PATH / "*.parquet"))  # fallback parquet

print("Filas:", df.count(), "| Columnas:", len(df.columns))

print("Schema df:")
df.printSchema()

Filas: 2682447 | Columnas: 26
Schema df:
root
 |-- fecha: string (nullable = true)
 |-- indicativo: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- provincia: string (nullable = true)
 |-- altitud: string (nullable = true)
 |-- tmed: string (nullable = true)
 |-- prec: string (nullable = true)
 |-- tmin: string (nullable = true)
 |-- horatmin: string (nullable = true)
 |-- tmax: string (nullable = true)
 |-- horatmax: string (nullable = true)
 |-- hrMax: string (nullable = true)
 |-- horaHrMax: string (nullable = true)
 |-- hrMin: string (nullable = true)
 |-- horaHrMin: string (nullable = true)
 |-- hrMedia: string (nullable = true)
 |-- dir: string (nullable = true)
 |-- velmedia: string (nullable = true)
 |-- racha: string (nullable = true)
 |-- horaracha: string (nullable = true)
 |-- presMax: string (nullable = true)
 |-- horaPresMax: string (nullable = true)
 |-- presMin: string (nullable = true)
 |-- horaPresMin: string (nullable = true)
 |-- sol: string (nul

## A. Datos generales
- A.1 Número de registros totales en el dataset
- A.2 Check de duplicados (`fecha` + `indicativo`)
- A.3 Número de registros por estación (cobertura)
  - Clasificación de estaciones por cobertura:
    - Alta (>90% de días con registros)
    - Media (80–90%)
    - Baja (<80%)
  - Evaluación de si el rango alto (90–100%) es aceptable
- A.4 Evaluación de estaciones de cobertura media y baja:
  - Distribución espacial (ubicación en el territorio)
  - Distribución temporal (¿faltantes dispersos o concentrados en periodos específicos?)
- A.5 Número de registros por estación y por año (con foco en estaciones de cobertura alta)

#### A.1 Número de registros totales en el dataset
- Numero de registros presentes en el dataset
- Numero de registros esperados en total

In [10]:
from pyspark.sql.functions import col, to_date, min as Fmin, max as Fmax
total_registros = df.count()
print("Total de registros en el dataset:", f"{total_registros:,}")

# Rango de fechas del dataset
if "fecha" in df.columns:
    df = df.withColumn("fecha", to_date(col("fecha")))
    start_date = df.select(Fmin("fecha")).first()[0]
    end_date = df.select(Fmax("fecha")).first()[0]
print(f"Rango fechas: {start_date} → {end_date}")

# Total estaciones
total_estaciones = df.select("indicativo").distinct().count()
print("Total de estaciones:", f"{total_estaciones:,}")

Total de registros en el dataset: 2,682,447
Rango fechas: 2017-01-01 → 2025-06-30
Total de estaciones: 918


In [8]:
from pyspark.sql.functions import col, to_date, year, count, lit, min as Fmin, max as Fmax, avg, stddev
from datetime import date

# Asegurar que 'fecha' es de tipo DATE
if "fecha" in df.columns:
    df = df.withColumn("fecha", to_date(col("fecha")))

# Parámetros del análisis (rango real del dataset)
START_DATE = date(2017, 1, 1)
END_DATE   = date(2025, 6, 30)

# Máximo de registros esperados por estación = días entre START_DATE y END_DATE (incluidos)
EXPECTED_MAX_PER_STATION = (END_DATE - START_DATE).days + 1

print(f"Rango analizado: {START_DATE} → {END_DATE}")
print("Registros esperados por estación:", EXPECTED_MAX_PER_STATION)

# Total esperado
expected_total = total_estaciones * EXPECTED_MAX_PER_STATION
print("Total esperado (estaciones × registros por estación):", f"{expected_total:,}")

Rango analizado: 2017-01-01 → 2025-06-30
Registros esperados por estación: 3103
Total esperado (estaciones × registros por estación): 2,848,554


El numero total de registros en el dataset es **2,682,447**. El total esperado es **2,848,554** registros, lo que indica que hay **166,107 registros menos de los esperados**.